# 1. 텍스트 데이터 다루기
## 1.1 자연어와 프로그래밍 언어
####  형식 언어 이론상 자연어와 프로그래밍 언어의 차이     
- 자연어 -  문맥 의존 문법(Context-sensitive Grammar)
- 프로그래밍 언어 - 문맥 자유 문법(Context-free Grammar)

프로그래밍 언어는 파서가 존재해 문법에 맞게 파싱을 하면 의도대로 동작한다(문맥 자유 문법을 따르기 때문). 그러나 자연어는 문맥에 따라 여러 가지로 해석이 가능한 경우가 있으며 현실 세계에 대한 지식을 통해 그 의미를 이해해야 한다. 

## 1.2 전처리: 자연어의 노이즈 제거
앞으로 다루게 될 언어모델(Language Model)은 단어가 출현하게 될 확률 모델이다. 언어 출현 확률의 일정한 패턴을 통계적으로 딥러닝을 이용해 학습시키기 때문에 문법이나 언어 사용 관습의 일정한 패턴을 따른 텍스트 데이터가 이상적이다. 그러나 띄어쓰기, 맞춤법, 약어 사용 등 일상어의 변형은 자연어처리 모델 입장에서는 노이즈이다. 노이즈를 해결하기 위해 소설책이나 신문 기사 같이 1) 맞춤법이 비교적 정확하고 노이즈가 적은 데이터를 사용하거나 여러 테크닉으로 2) 노이즈를 제거한 후 데이터를 사용하기도 한다. (하지만 데이터의 양이 많으면 노이즈가 상쇄될 수 있다.)

#### 노이즈의 종류     
1. 불완전한 문장으로 구성된 대화    
2. 문장의 길이가 너무 길거나 짧은 경우     
짧은 문장은 의미가 없거나 리액션, 긴 문장은 대화와 관계 없는 문장일 수 있음.
3. 채팅 데이터에서 문장 시간 간격이 너무 긴 경우      
서로의 말이 얽히거나 말의 텀이 길면 연속된 대화로 보기 어려움.
4. 바람직하지 않은 문장의 사용    
욕설이나 오타의 비율이 높은 문장.

### 대표적인 세 가지 노이즈 유형 
문장부호, 대문자, 특수문자

#### 1) 문장부호: 문장 부호 양쪽에 공백을 추가
그러나 노이즈 제거에 큰 영향을 주지 않음

#### 2) 대소문자: 모든 단어를 소문자로 바꾸기

#### 3) 특수문자: 정규표현식으로 제거
참고: [정규표현식 정리](https://hamait.tistory.com/342)

- 정규 표현식 도식화 사이트
1) RegExr(https://regexr.com/)      
2) REGEXPER(https://regexper.com/)

[^a-zA-Z.,?!]    
- ^: 반대
- 문장 내 알파벳 모두와 '.', ',', '?', '!'를 제외한 특수문자는 모두 빈 칸으로 대체한다.  

## 1.3 분산표현

### 단어의 희소 표현과 분산 표현
임베딩 레이어(Embedding Layer)를 통해 단어의 __분산 표현(distributed representation)__ 를 구현할 수 있다.

#### 희소 표현
희소 표현(Sparse representation): 벡터 또는 행렬의 값이 대부분 0으로 표현되는 방법, 원-핫 벡터는 희소 백터이다.  

- 희소 표현 방식의 단어 사전
```python
{
    //      [성별, 연령, 과일, 색깔]
      남자: [-1.0, 0.0, 0.0, 0.0],
      여자: [1.0, 0.0, 0.0, 0.0],
      사과: [0.0, 0.0, 1.0, 0.5],   // 빨갛게 잘 익은 사과
      바나나: [0.0, 0.0, 1.0, -0.5] // 노랗게 잘 익은 바나나
}
```
- 문제점 
1. 단어의 수(혹은 속성의 종류)가 늘어나면 0으로 채워진 고차원의 벡터가 만들어지므로 불필요한 메모리와 연산량이 낭비된다. 
2. 단어들 간의 의미적 유사도를 계산할 수 없다. (의미 속성을 공유하지 않는 희소 표현 워드 벡터의 경우)

>[코사인 유사도](https://wikidocs.net/24603)       
두 고차원 벡터의 유사도는 __코사인 유사도(Cosine Similarity)__ 를 통해 구할 수 있다. 코사인 유사도는 두 벡터 간 코사인 각도를 이용해 구할 수 있는 두 벡터의 유사도이다. 두 벡터의 방향이 완전히 동일하면 1, 90도면 0, 반대이면 -1의 값을 갖는다. 코사인 유사도는 -1 이상 1 이하의 값을 가지며 __1에 가까울수록 유사도가 높다__. 코사인 유사도는 문서의 길이가 다른 상황에서 비교적 공정한 비교를 할 수 있도록 도와준다. 
>
>![](https://images.velog.io/images/guide333/post/ed7acbf9-d367-49cd-ba95-bdc1dc6cc9a7/Screenshot%20from%202021-04-05%2019-38-24.png)

#### 밀집 표현
희소 표현과 반대되는 밀집 표현(dense representation)은 사용자가 설정한 값으로 모든 단어의 벡터 표현의 차원을 맞추고, 그 과정에서 실수값을 가진다. 

예> 희소표현: 강아지 = [ 0 0 0 0 1 0 0 0 0 0 ... 중략 ... 0] # 1 뒤의 0의 수는 9995개. 차원은 10,000        
    밀집 표현: 강아지 = [0.2 1.8 1.1 -2.1 1.1 2.8 ... 중략 ...] # 이 벡터의 차원은 128, 모든 값이 실수
    
```python
# 10,000개의 단어를 128 차원의 속성으로 표현 코드
embedding_layer = tf.keras.layers.Embedding(input_dim=10000, output_dim=128)
```

#### 분산 표현
단어를 밀집 행렬로 표현하는 것을 워드 임베딩(word embedding) 혹은 단어의 분산 표현(distributed representation)이라고 한다. 또한 밀집 벡터를 __임베딩 벡터(Embedding vector)__ 라고도 한다. Embedding 레이어를 사용해 각 단어가 몇 차원의 속성을 가질지 정의하는 방식으로 단어의 분산 표현을 구현할 수 있다. 단어의 분산 표현에는 정의할 수 없는 추상적인 속성들이 128차원 안에 골고루 분산되어 표현된다. 적절히 훈련된 분산 표현 모델을 통해 단어 간 의미 유사도를 계산하거나 feature로 삼아 복잡한 자연어처리 모델을 훈련시킬 수 있다. 

Embedding 레이어를 사용해 구현한 분산 표현은 컴퓨터 입장에서는 단어 사전이지만 컴퓨터는 전혀 엉뚱한 단어로 해석하거나 혹은 사전에서 그 단어를 찾지 못해 당황하는 일이 생길 수 있다. 이를 해결하기 위해서 토큰화를 한다. 

## 1.4 토큰화
토큰화(Tokenization): 주어진 코퍼스에서 토큰(Token)이라 불리는 단어로 나누는 작업. 보통 의미 있는 단위로 토큰을 정의한다. 

### 공백 기반 토큰화
자연어의 노이즈를 제거하는 방법 중 하나로 문장부호 양 옆에 공백을 추가함.```split()``` 함수 사용하면 자동으로 토큰을 List로 만들어 준다.      
단점: 같은 단어인 days와 day도 따로 저장된다. 

### 형태소 기반 토큰화
한국어 문장의 경우 공백 기준으로 토큰화하면 의도치 않은 변형이 많이 일어난다. 이를 해결하기 위해 __형태소__ 기반 토큰화이다. 여기서 형태소는 "뜻을 가진 가장 작은 말의 단위"이다. 형태소 문해는 문장에서 위치나 문맥에 따라 달라진다. 

- 대표적인 한국어 형태소 분석기
1. [KoNLPy: 파이썬 한국어 NLP - KoNLPy 0.4.3 documentation](https://konlpy-ko.readthedocs.io/ko/v0.4.3/): 5개의 형태소 분석 class 포함
2. [kakao/khaiii](https://github.com/kakao/khaiii): 윈도우 지원 불가

>[한국어 형태소 분석기 성능 비교](https://iostream.tistory.com/144)      
형태소 분석기들은 특수한 문장 처리 성능, 속도 측면에서 차이를 보인다.        
> 형태소 분석기가 처리하기 힘든 유형 3가지: 띄어쓰기가 없는 문장, 자소 분리 및 오탈자가 포함된 문장, 긴 문장   
> 사용할 데이터의 특성이나 개발 환경에 따라 적합한 형태소 분석기를 고려해야 한다. 

### 사전에 없는 단어의 문제
지금까지 배운 토큰화 기법은 의미를 가지는 단위로 토큰을 생성한다. 그러나 이 기법들은 자주 등장하는 상위 N개의 단어만 사용하고 나머지는 ```<unk>``` 같은 특수한 토큰(Unknown Token)으로 치환하기 때문에 새로운 단어 나오면 OOV(Out-Of-Vocabulary) 문제가 발생한다. 이를 해결하는 시도가 Wordpiece Model이다. 


## 1.5 Wordpiece Model(WPM)
한 단어를 여러 개의 subword의 집합으로 보는 방법

### BPE(Byte Pair Encoding)
1994년 데이터 압축을 위해 생겨난 BPE는 가장 많이 등장하는 바이트 쌍(Byte Pair)을 새로운 단어로 치환하여 압축하는 작업을 반복한다. 이를 토큰화에 적용하기 시작한 것은 2015년이다. 자주 등장하는 문자 쌍을 합쳐 접두어나 접미어의 의미를 캐치할 수 있고, 처음 등장하는 단어는 문자의 조합으로 나타내어 OOV 문제를 해결할 수 있다. 

논문: [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf)

장점: 큰 데이터를 OOV 문제 없이 원하는 크기의 사전으로 정의할 수 있다. 많은 데이터도 적은 단어의 개수로 줄여주므로 정확도는 높고 메모리도 절약할 수 있다. 그러나 어떤 기준으로 결합할 것이냐에 따라 결과가 달라진다. 

### WordPiece Model(WPM)
구글에서 BPE를 변형해 제안한 알고리즘이다. 

1. 공백 복원을 위해 단어의 시작 부분에 언더바(\_)를 추가: 모든 토큰을 합친 후, 언더바\_를 공백으로 치환.
2. 가능도(Liklihood)를 증가시키는 방향으로 문자 쌍을 합친다. : [JAPANESE AND KOREAN VOICE SEARCH](https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf)의 3, 4절에 2번 내용이 자세하게 나옴. (이 부분은 따로 정리할 것) 

- WPM의 장점
1. 조사, 어미 등의 활용이 많고 복잡한 한국어 같은 모델의 토크나이저로 WPM이 좋은 대안
2. language-neutral, general한 기법

- [확률(probability)과 가능도(likelihood) 그리고 최대우도추정(likelihood maximazation)](https://jjangjjong.tistory.com/41)

저자들은 Katz's Back-off Model 을 통해 토큰화된 사전으로부터 결합할 단어를 선정한다. Katz's Back-off Model은 LM에서 등장하지 않은 단어에 대해 확률을 할당할 수 있는 모델(흔히 사용되는 개념은 아니므로 수식을 이해할 필요는 없음)

![](https://images.velog.io/images/guide333/post/ac299e8f-d52b-4ea0-9653-495ea343ea4d/Screenshot%20from%202021-04-05%2017-15-26.png)

WPM은 공개되어 있지 않고, 구글의 SentencePiece 라이브러리를 통해 고성능의 BPE를 사용할 수 있다. 전처리 과정도 포함되어 있어서 편리하다. 

- 사용법: [google/sentencepiece](https://github.com/google/sentencepiece)


## 1.6 유사도 연산 알고리즘 3가지

### Word2Vec
Word2Vec은 동시에 등장하는 단어끼리는 연관성이 있다는 알고리즘이며, 두 가지 방식, 즉 CBOW, Skip-gram이 있다. Word2Vec은 은닉층이 1개 이므로 딥러닝 모델이 아니며 CBOW가 논리적이지만 실제 실험 결과에서는 Skip-gram이 더 좋은 결과를 나타낸다. 

### FastText
[한국어를 위한 어휘 임베딩의 개발 -1-](https://brunch.co.kr/@learning/7)

FastText는 한 단어를 n-gram의 집합이라고 보고 단어를 쪼개어 각 n-gram에 할당된 Embedding의 평균값을 사용하였다.

### ELMo -  the 1st Contextualized Word Embedding
기존의 Word Embedding 알고리즘은 고정적이므로 동음이의어를 처리할 수 없다(동음이의어의 워드 벡터값이 동일). 자연어를 이해하려면 단어와 주변 단어 배치의 맥락이 함께 고려되는 Word Embedding이 필요하다. 이를 Contextualized Word Embedding라고 한다. 

2018년 ELMo는 단어와 그 주변 단어 정보를 사용해 Embedding을 구축하는 개념을 처음 소개한 첫번째 Contextualized Word Embedding 모델이다. 

[전이 학습 기반 NLP (1): ELMo](https://brunch.co.kr/@learning/12)

양방향 LSTM을 활용한 ELMo 모델에서 Contextual Word Embedding 벡터는 기존 어휘 임베딩, 순방향 LSTM의 hidden state vector, 역방향 LSTM의 hidden state vector를 concate한 벡터이다. 

- 1.6은 따로 정리!! 

- 참고 자료: AIFFEL 대전 노드, [딥러닝을 이용한 자연어 처리(WIKIDOCS)](https://wikidocs.net/book/2155)
